In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
import dill
import sys
sys.path.append('../spock/')
from modelfitting import ROC_curve
try:
    plt.style.use('paper')
except:
    pass
import pandas as pd
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
from sklearn.metrics import roc_curve, auc 
from sklearn import metrics

%matplotlib inline

In [4]:
dfres = pd.read_csv('resPratios.csv', index_col=0)
dfrand = pd.read_csv('randomPratios.csv', index_col=0)

# Models trained on resonant dataset

Test on full 25,000 random systems

In [14]:
thresh=0.1
models = {'N-body':'nbody', 'SPOCK':'spock', 'MEGNO':'megno', 'AMD':'AMD', 'Hill':'hill'}

fig = plt.figure(figsize=(8,16))
gs = gridspec.GridSpec(2, 1, figure=fig)
gs.update(wspace=0.0)

stable = dfrand['instability_time'] > 9.99e8
ax1 = plt.subplot(gs[0])
ax1.set_rasterization_zorder(1)
ax1.scatter(dfrand['beta12'][~stable], dfrand['beta23'][~stable], c = 'r', s=2, alpha=0.3, zorder=0)
ax1.scatter(dfrand['beta12'][stable], dfrand['beta23'][stable], c = 'b', s=2, alpha=0.3, zorder=0)

ax1.set_xlim([3,30])
ax1.set_ylim([3,30])
ax1.set_xlabel('Inner Planet Pair Separation (Hill radii)')#r'$\Delta_{12}$')
ax1.set_ylabel('Outer Planet Pair Separation (Hill radii)')#'r'$\Delta_{23}$')
ax1.locator_params(nbins=6)
ax1.set_title('Random Dataset')


Nstable = stable.sum()
Nunstable = (~stable).sum()
legend_elements = [Line2D([0], [0], marker='o', lw=0, c='k', markerfacecolor='r', label='Unstable ({0} systems)'.format(Nunstable), markersize=10),
                   Line2D([0], [0], marker='o', lw=0, c='k', label='Stable ({0} systems)'.format(Nstable), markerfacecolor='b', markersize=10)]
ax1.legend(handles=legend_elements, loc='upper right')

testing_dataset = 'random'

ax = plt.subplot(gs[1])

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.0])
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')


for name, filename in models.items():
    model, features, featureargs, featurefolder = dill.load(open('../models/'+filename+'.pkl', "rb"))
    testingdatafolder = '../training_data/random/'+featurefolder
    dataset = pd.read_csv(testingdatafolder+"trainingdata.csv", index_col = 0)
    labels = pd.read_csv(testingdatafolder+"labels.csv", index_col=0)
    y = labels['Stable']
    if features is None:
        features = dataset.columns.values
    X = dataset[features]
    preds = model.predict_proba(X)[:,1]
    fpr, tpr, ROCthresholds = roc_curve(y, preds)
    roc_auc = metrics.roc_auc_score(y, preds)
    threshindex = np.where(fpr > thresh)[0][0]
    print(name, "AUC = {0:.3f} Thresh = {1:.3f}".format(roc_auc, ROCthresholds[threshindex]),  "TPR = {0:.3f}".format(tpr[threshindex]), "FPR = {0:.3f}".format(fpr[threshindex]))
    ax.plot(fpr, tpr, label='{0} ({1:.2f})'.format(name, roc_auc))

ax.plot(np.linspace(0,1), np.linspace(0,1), 'k--', label='Random')
ax.text(0.35, 0.2, "Random", fontsize=20)
ax.text(0.355, 0.15, "Dataset", fontsize=20)


ax.legend()
fig.savefig('randomperformance.pdf', bbox_inches='tight')

N-body AUC = 0.991 Thresh = 0.173 TPR = 0.998 FPR = 0.107
SPOCK AUC = 0.972 Thresh = 0.597 TPR = 0.930 FPR = 0.100
MEGNO AUC = 0.944 Thresh = 0.966 TPR = 0.865 FPR = 0.101
AMD AUC = 0.897 Thresh = 0.926 TPR = 0.716 FPR = 0.100
Hill AUC = 0.817 Thresh = 0.920 TPR = 0.378 FPR = 0.100


In [8]:
FPRthresh = 0.1

for name, filename in models.items():
    model, features, featureargs, featurefolder = dill.load(open('../models/'+filename+'.pkl', "rb"))
    testingdatafolder = '../training_data/random/'+featurefolder
    roc_auc, fpr, tpr, ROCthresholds = ROC_curve(testingdatafolder, model, features,  filtertimes=False)
    threshindex = np.where(fpr > FPRthresh)[0][0]
    print(name, "thresh = {0:.3f}".format(ROCthresholds[threshindex]),  "TPR = {0:.3f}".format(tpr[threshindex]), "FPR = {0:.3f}".format(fpr[threshindex]))

N-body thresh = 0.173 TPR = 0.996 FPR = 0.101
SPOCK thresh = 0.527 TPR = 0.943 FPR = 0.101
MEGNO thresh = 0.963 TPR = 0.889 FPR = 0.102
AMD thresh = 0.923 TPR = 0.734 FPR = 0.100
Hill thresh = 0.917 TPR = 0.429 FPR = 0.100


In [9]:
TPRthresh = 0.9

for name, filename in models.items():
    model, features, featureargs, featurefolder = dill.load(open('../models/'+filename+'.pkl', "rb"))
    testingdatafolder = '../training_data/random/'+featurefolder
    roc_auc, fpr, tpr, ROCthresholds = ROC_curve(testingdatafolder, model, features)
    threshindex = np.where(tpr > TPRthresh)[0][0]
    print(name, "thresh = {0:.3f}".format(ROCthresholds[threshindex]),  "TPR = {0:.3f}".format(tpr[threshindex]), "FPR = {0:.3f}".format(fpr[threshindex]))

N-body thresh = 0.996 TPR = 0.978 FPR = 0.014
SPOCK thresh = 0.776 TPR = 0.901 FPR = 0.073
MEGNO thresh = 0.960 TPR = 0.902 FPR = 0.108
AMD thresh = 0.842 TPR = 0.901 FPR = 0.290
Hill thresh = 0.788 TPR = 0.900 FPR = 0.404
